In [1]:
import pygame as p
%run ChessEngine.ipynb

WIDTH = HEIGHT = 512
CELLS = 8
SQ_SIZE = HEIGHT // CELLS
IMAGES = {}
GREEN = (76, 153, 0)
WHITE = (229, 255, 204)

pygame 2.5.2 (SDL 2.28.3, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def loadImages():
    pieces = ['bR','bN','bB','bQ','bK','bp','wR','wN','wB','wQ','wK','wp']
    for piece in pieces:
        IMAGES[piece] = p.image.load("images/" + piece + ".png")

In [3]:
def main():
    p.init()
    screen = p.display.set_mode((WIDTH, HEIGHT))
    running = True
    gs = GameState()
    # print(gs.board)
    loadImages()
    
    while running:
        for event in p.event.get():
            if event.type == p.QUIT:
                running = False
        drawGameState(screen, gs)
        p.display.flip() 
    p.quit()

In [4]:
def drawGameState(screen, gs):
    drawSquares(screen)
    drawPieces(screen, gs.board)
    

In [5]:
def drawSquares(screen):
    colors = [p.Color(WHITE),p.Color(GREEN)]
    for row in range(CELLS):
        for col in range(CELLS):
            color = colors[(row + col) % 2]
            p.draw.rect(screen, color, p.Rect(col * SQ_SIZE, row * SQ_SIZE, SQ_SIZE, SQ_SIZE))

In [6]:
def drawPieces(screen, board):
    for row in range(CELLS):
        for col in range(CELLS):
            piece = board[row][col]
            if piece != "--":
                screen.blit(IMAGES[piece], p.Rect(col * SQ_SIZE, row * SQ_SIZE, SQ_SIZE, SQ_SIZE))
                

In [7]:
if __name__ == "__main__":
    main()